In [4]:
#Importar librerías
import math
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from IPython.display import HTML, display

In [5]:
#Leer el archivo con los datos
data = pd.read_csv('./data/msd_genre_dataset.csv')
data=data.drop(['artist_name', 'track_id', 'title'], axis=1)
data1=data.loc[data['genre'] == 'jazz and blues']
data2=data.loc[data['genre'] == 'soul and reggae']
data= pd.concat([data1, data2], axis= 0)
print(data)

                 genre  loudness    tempo  time_signature  key  mode  \
48942   jazz and blues   -18.996   89.147               1    0     1   
48943   jazz and blues   -19.347  125.825               4    2     1   
48944   jazz and blues    -9.472  121.707               4    2     1   
48945   jazz and blues   -14.406   83.012               5    0     1   
48946   jazz and blues    -9.289   76.013               4    7     0   
48947   jazz and blues   -10.003  143.751               4    0     0   
48948   jazz and blues   -11.161  122.412               4    0     1   
48949   jazz and blues   -12.785  244.366               4   10     1   
48950   jazz and blues    -5.834  143.886               1    9     0   
48951   jazz and blues   -12.174   74.642               1    2     0   
48952   jazz and blues    -7.535  139.480               3   11     1   
48953   jazz and blues   -11.945  101.866               7    0     1   
48954   jazz and blues   -11.294  122.794               4    7  

Si se utilizan 600 datos para realizar las validaciones y evaluaciones del modelo respectivamente, entonces la confianza con la que el error estimado esté máximo a 0.1 de diferencia del error real es de:

In [6]:
confianza=1-2*math.exp(-2*0.05**2*600)
print(confianza)

0.9004258632642721


Como se puede observar esta confianza es del 90%. Lo cual significa que usar 600 datos es un buen estimativo para estar máximo a 0.1 del error.
Se parten los datos en entrenamiento y test

In [7]:
#La variable dependiente es la última columna, las independientes son las anteriores.
#Se normaliza las independientes y se convierte nuevamente en un DataFrame de Pandas
x= pd.DataFrame.from_records(preprocessing.normalize(data.loc[:, data.columns != 'genre']))
y= data.iloc[:,0:1]
le = preprocessing.LabelEncoder()
le.fit(["soul and reggae", "jazz and blues"])
y=pd.DataFrame(le.transform(y).astype(float).T)
#Se divide el archivo para entrenamiento y test. Se reserven 10000 datos para test
xTrainT, xTest, yTrainT, yTest = train_test_split(x, y, test_size = 600, random_state = 0)
print(xTrainT)

            0         1         2         3         4         5         6   \
3989 -0.007746  0.030671  0.001081  0.001351  0.000270  0.038641  0.004707   
2962 -0.003219  0.024831  0.000703  0.000527  0.000176  0.142768  0.005611   
6885 -0.003161  0.018672  0.001030  0.000000  0.000206  0.033097  0.007638   
8003 -0.003776  0.038427  0.001272  0.000000  0.000000  0.062554  0.013290   
2528 -0.002559  0.017013  0.000765  0.000000  0.000191  0.048355  0.007027   
6877 -0.001435  0.018808  0.000564  0.000000  0.000141  0.024617  0.005608   
7107 -0.001558  0.019494  0.000460  0.000805  0.000115  0.023422  0.004027   
6453 -0.002304  0.016158  0.000799  0.001598  0.000200  0.051060  0.007661   
5386 -0.002765  0.051769  0.001238  0.003096  0.000000  0.068269  0.013342   
5329 -0.001658  0.032536  0.000746  0.001740  0.000249  0.045790  0.011759   
2260 -0.003426  0.054033  0.001448  0.002533  0.000000  0.063492  0.015787   
3913 -0.001795  0.018823  0.000576  0.000863  0.000144  0.034533

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
#Se concatenan los datos de test
newData= pd.concat([xTrainT,yTrainT], axis = 1)
xTrain, xValidation, yTrain, yValidation = train_test_split(xTrainT, yTrainT, test_size = 600, random_state = 0)
print(yTrain)

        0
8064  1.0
6747  1.0
6382  1.0
470   0.0
5518  1.0
2642  0.0
7383  1.0
4878  1.0
2682  0.0
751   0.0
2210  0.0
7052  1.0
2010  0.0
339   0.0
5905  1.0
7381  1.0
1681  0.0
7374  1.0
1944  0.0
3769  0.0
8146  1.0
1636  0.0
6296  1.0
4281  0.0
369   0.0
4500  1.0
3256  0.0
4635  1.0
7322  1.0
3564  0.0
...   ...
5704  1.0
2176  0.0
367   0.0
4140  0.0
5700  1.0
2279  0.0
7842  1.0
1206  0.0
1427  0.0
4016  0.0
770   0.0
1615  0.0
7840  1.0
1349  0.0
4787  1.0
5116  1.0
5670  1.0
1320  0.0
5624  1.0
1316  0.0
2722  0.0
1433  0.0
4652  1.0
3919  0.0
5981  1.0
3469  0.0
2636  0.0
3181  0.0
4668  1.0
1763  0.0

[7150 rows x 1 columns]


In [234]:
#Función para entrenar un Árboles de decisión con Adaboost
def trainTrees(min_samples_split,n_estimators):
    clf = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=min_samples_split),n_estimators=n_estimators)
    clf.fit(xTrain.values.tolist(), yTrain.values.tolist())   
    return [clf,clf.score(xValidation,yValidation)]

In [235]:
#En el primer arreglo se guardarán los errores y en el segundo los clasificadores entrenados
errorsLayers=[]
trees=[]

In [236]:
#Se iterarán por gammas y C's en un rango logarítmico. Este rango fue hallado empíricamente
min_samples_range = np.linspace(2,11,10).astype(int).tolist()
estimators_range = np.linspace(70,170,10).astype(int).tolist()

In [237]:
#Se itera por los gammas y los C's y se calcula el porcentaje de éxito de cada svm con los datos de validación
for i in min_samples_range:
    errorsTemp=[]
    treesTemp=[]
    for j in estimators_range:
        tree=trainTrees(i,j)
        treesTemp.append(tree[0])
        errorsTemp.append(tree[1])
    errorsLayers.append(errorsTemp)
    trees.append(treesTemp)
errorsLayers=np.array(errorsLayers)
trees=np.array(trees)

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [238]:
print(errorsLayers)

[[0.78       0.78166667 0.78       0.77833333 0.78333333 0.77166667
  0.78833333 0.79       0.775      0.78      ]
 [0.87166667 0.87       0.875      0.87333333 0.86833333 0.86333333
  0.88333333 0.87666667 0.87833333 0.875     ]
 [0.86       0.86166667 0.86666667 0.86666667 0.87666667 0.87166667
  0.87       0.88333333 0.88166667 0.87166667]
 [0.865      0.86333333 0.86666667 0.88333333 0.87666667 0.87
  0.86833333 0.87833333 0.86666667 0.875     ]
 [0.86833333 0.87166667 0.875      0.87833333 0.86333333 0.875
  0.87       0.87666667 0.885      0.88666667]
 [0.85666667 0.875      0.87833333 0.86       0.87833333 0.87333333
  0.875      0.88333333 0.87       0.88      ]
 [0.86666667 0.88166667 0.88333333 0.875      0.875      0.875
  0.88333333 0.875      0.87333333 0.86833333]
 [0.86       0.89333333 0.87       0.88       0.86833333 0.88666667
  0.87166667 0.86666667 0.88666667 0.87333333]
 [0.865      0.87       0.87166667 0.87166667 0.89       0.875
  0.875      0.88333333 0.88     

In [239]:
#Algoritmo para encontrar los índices del elemento más grande de un arreglo 2D, en caso de varias ocurrencias retorna la última
def findMaxIndexes(arr):
    maxC=0
    maxIndex=[-1,-1]
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            if arr[i][j] is not None and arr[i][j]>=maxC:
                maxIndex=[i,j]
                maxC=arr[i][j]
    return maxIndex

In [240]:
#Se utiliza el algoritmo anterior para encontrar la red de mejor resultado
[ind1,ind2]=findMaxIndexes(errorsLayers)
print(ind1,ind2)
print(errorsLayers[ind1][ind2])
print(min_samples_range[ind1])
print(estimators_range[ind2])

7 1
0.8933333333333333
9
81


In [10]:
#Se entrena un árbol de decisión ahora sin adaboost
clf = DecisionTreeClassifier(min_samples_split=9)
clf.fit(xTrain.values.tolist(), yTrain.values.tolist())   
print(clf.score(xValidation,yValidation))

0.775


In [241]:
#Se muestran los resultados en formato de tabla
df=pd.DataFrame(errorsLayers)
display(HTML(df.to_html()))

En el punto anterior se observa que un árbol de decisión con adaboost con mínimo 9 muestras para subdividir un nodo y 81 estimadores de adaboost fue el que obtuvo el mínimo error, esta tasa de éxito se observa que corresponde a 0.893333. Esta tasa de éxito es superior a la obtenida con la red neuronal en el reto 5 y ligeramente superior a la del svm en el reto 7. Esto prueba que incluso un modelo tan sencillo como es el de un árbol de decisión puede ver incrementado su tasa de éxito significativamente con Boosting

In [242]:
bestTree= trees[ind1][ind2]
#El svm de mejor resultado se entrena con los datos de entrenamiento totales
bestTree.fit(xTrainT, yTrainT)
#Se prueba con los datos de prueba
bestTree.score(xTest,yTest)

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8566666666666667

Se puede observar que la tasa de éxito de clasificación es de 0.856666. Esta tasa de éxito es menor que la tasa de éxito con menos datos, esto puede deberse a que los datos finales de testing no encajaban con lo que el modelo se entrenó. De acuerdo al cálculo anterior, se podría afirmar que con una confianza del 90% la tasa de éxito mínima es de 0.8466666666666667.

Se puede concluir que un algoritmo de Boosting es útil para mejorar la tasa de clasificación exitosa de un algoritmo tal como un árbol de decisión. Se puede observar por ejemplo que la tasa de éxito del algoritmo con adaboost aumenta en más de 0.1 respecto a tener el árbol de decisión solo. Es importante tener en cuenta que la prácticidad de adaboost también recide en que el árbol de decisión es rápido en su entrenamiento, por lo que el tiempo adicional que le toma a adaboost entrenar no es una complicación significativa para el problema tratado